In [ ]:
from ultralytics import YOLO
from PIL import Image
from IPython.display import display
import pandas as pd
from collections import Counter

# Гиперпараметры

In [ ]:
DICT_CLASSES_INT = {'gun': 0, 'person': 1}
DICT_INT_CLASSES = {v:k for k, v in DICT_CLASSES_INT.items()}

# Список моделей.
MODEL_PARAMS = [
    {'EXPERIMENT_NAME': 'yolov8n_experiment5', 'MODEL_NAME': 'best.pt', 'IMGSZ': 640},
    {'EXPERIMENT_NAME': 'yolov8n_tune5', 'MODEL_NAME': 'best.pt', 'IMGSZ': 640},
]

# Создание путей до обученной модели.
PATH_TO_MODEL = lambda exp_name, model_name: f'../runs/detect/{exp_name}/weights/{model_name}'

# Данные для тестирования
PATH_TO_TEST_DATASET = '../dataset/valid/images/'

# Параметры для тестирования
COMPARE_BY_COUNT = True  # Сравнивать по количеству, иначе по conf(достоверности предсказания)
TEST_PARAMS = {
    'source': PATH_TO_TEST_DATASET,
    'save': False,
    'show': False,
    'verbose': False,

    'imgsz': 640,
    'conf': 0.25,  # порог достоверности обнаружения объекта | попробовать 0.2 потом
    'iou': 0.7,  # пересечение порога объединения (IOU)
}

Проверка GPU

In [ ]:
import ultralytics
ultralytics.checks()

In [ ]:
import torch

cuda_flag = torch.cuda.is_available()
device = 'cuda' if cuda_flag else 'cpu'
device

Необходимые функции

In [ ]:
from typing import Dict, Union, Tuple, List


def create_model_test_params(model_params: Dict[str, Union[str, int]]) -> Tuple[str, str]:
    """Создание параметров для тестирования конкретной модели."""
    path_to_model = PATH_TO_MODEL(model_params['EXPERIMENT_NAME'], model_params['MODEL_NAME'])

    test_params = TEST_PARAMS.copy()
    test_params['imgsz'] = model_params['IMGSZ']
    return path_to_model, test_params


def show_pred_img(preds: List, index_show: int) -> None:
    """Результат работы модели на одной картинке"""
    img = preds[index_show].plot()
    img = Image.fromarray(img[..., ::-1])
    display(img)


def get_classes_images(model, preds: List) -> Dict[str, List[Dict[str, Union[str, float]]]]:
    """Получаем классы, предсказанные одной моделью."""
    result = {}

    names = model.names
    for pred in preds:
        path = pred.path
        path = path.split('images')[-1][1:]                                             # TODO: возможно название картинки нужно менять.
        result[path] = []
        for i in range(len(pred.boxes.cls)):
            cls = pred.boxes.cls[i]
            conf = pred.boxes.conf[i]
            tmp_dict = {'class': names[int(cls)], 'conf': conf.item()}
            result[path].append(tmp_dict)
    return result


def create_df_predicts(model_result: Dict[str, Union[str, Dict]], filename_csv: str) -> pd.DataFrame:
    """Создание DataFrame и csv-файла для submission.

    model_result:
        {'path_to_model': str, 'class_images': {str: [{'class': str,'conf': float}, ], }} """

    lst_image_names = list(model_result['class_images'].keys())
    lst_image_class = []

    for lst in model_result['class_images'].values():

        # Считаем кол-во классов, определённых на изображении.
        dict_counter_classes = {key: 0 for key in DICT_CLASSES_INT.keys()}                     # TODO: возможно надо будет поменять.
        for dct in lst:
            dict_counter_classes[dct['class']] += 1 if COMPARE_BY_COUNT else dct['conf']

        # Названием метку изображению по бОльшему кол-ву найденных классов.
        result_class = Counter(dict_counter_classes).most_common()[0][0]
        # result_class = DICT_CLASSES_INT[result_class]                                        # Если нужно не str, а int
        lst_image_class.append(result_class)

    df = pd.DataFrame({'image_name': lst_image_names, 'class': lst_image_class})
    df.to_csv(filename_csv, index=False)
    print(f'CSV-файл сохранён: {filename_csv}')
    return df


def predict(model_param: dict, index_show: int = 0) -> Dict[str, Union[str, Dict]]:
    """Получение предсказаний от одной модели.
    
    returns:
        {'path_to_model': str, 'class_images': {str: [{'class': str,'conf': float}, ], }}"""
    path_to_model, test_params = create_model_test_params(model_param)
    
    model = YOLO(path_to_model)
    preds = model.predict(**test_params, device=device)

    # Вывод предсказанной картинки.
    print(f'Model = {path_to_model}')
    show_pred_img(preds, index_show)

    classes_images = get_classes_images(model, preds)
    model_result = {'path_to_model': path_to_model, 'class_images': classes_images}
    return model_result


def voting_predict(lst_dataframe_results: List[pd.DataFrame], filename_csv: str) -> Dict[str, str]:
    """Голосование моделей по количеству."""

    lst_image_names = lst_dataframe_results[0].image_name.values
    lst_image_class = []
    
    for i in range(lst_dataframe_results[0].shape[0]):
        dict_counter_classes = {key: 0 for key in DICT_CLASSES_INT.keys()}
        for df in lst_dataframe_results:
            tmp_class = df['class'].values[i]
            # tmp_class = DICT_INT_CLASSES[tmp_class]                                         # Если в df классы представлены числами, а не строками.
            dict_counter_classes[tmp_class] += 1

        result_class = Counter(dict_counter_classes).most_common()[0][0]
        # result_class = DICT_CLASSES_INT[result_class]                                        # Если нужно не str, а int
        lst_image_class.append(result_class)
        
    df = pd.DataFrame({'image_name': lst_image_names, 'class': lst_image_class})
    df.to_csv(filename_csv, index=False)
    print(f'CSV-файл сохранён: {filename_csv}')
    return df

# Yolo Ensemble-Voting Test

In [ ]:
lst_dataframe_results = []

for model_param in MODEL_PARAMS:
    model_result = predict(model_param)

    # Сохранение результата одной модели
    filename_csv = f'{model_param["EXPERIMENT_NAME"]}_{model_param["MODEL_NAME"].split(".")[0]}.csv'
    df = create_df_predicts(model_result, filename_csv)
    
    lst_dataframe_results.append(df)

Пример созданного DataFrame

In [ ]:
example_df = lst_dataframe_results[0]
example_df

Создаём ансамбль голосования моделей

In [ ]:
filename_csv = 'voting.csv'
voting_predict(lst_dataframe_results, filename_csv)